In [2]:
import os
import json
import torch
import numpy as np
import pandas as pd
from PIL import Image
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from transformers import CLIPModel, CLIPProcessor, CLIPTokenizer
from transformers import SiglipModel, SiglipProcessor

In [6]:
# Step 3: Extract the dataset from the zip file
import zipfile
import os

# Path to your zip file in Google Drive
zip_path = '/content/drive/MyDrive/facebookds.zip'  # Update with your zip file path
extract_path = '/content/dataset'  # Directory to extract files

# Create extraction directory if it doesn't exist
if not os.path.exists(extract_path):
    os.makedirs(extract_path)

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Dataset extracted to: {extract_path}")

Dataset extracted to: /content/dataset


In [4]:
from transformers import AutoProcessor


In [3]:
    siglip_model = SiglipModel.from_pretrained("google/siglip-base-patch16-224")
    siglip_processor = SiglipProcessor.from_pretrained("google/siglip-base-patch16-224")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/813M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/368 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/711 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/409 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

In [7]:
import torch.nn as nn
from transformers import AutoModel

class SigLIPClassifier(nn.Module):
    def __init__(self, model_name):
        super(SigLIPClassifier, self).__init__()
        self.siglip = AutoModel.from_pretrained(model_name)

        # Access vision_config through the main model's config
        hidden_dim = self.siglip.config.vision_config.hidden_size  # Corrected line

        self.fc = nn.Linear(hidden_dim, 1)  # Binary classification

    def forward(self, inputs):
        outputs = self.siglip(**inputs).last_hidden_state[:, 0, :]  # CLS token
        logits = self.fc(outputs)
        return torch.sigmoid(logits)


In [8]:

# Load model
model_name = "google/siglip-base-patch16-256"
processor = AutoProcessor.from_pretrained(model_name)
model = SigLIPClassifier(model_name).to("cuda")

preprocessor_config.json:   0%|          | 0.00/368 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/711 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/409 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/322 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/813M [00:00<?, ?B/s]

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
import json
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader


# Load dataset JSON
with open("/content/balanced_new.json", "r") as f:
    dataset = json.load(f)

class MemeDataset(Dataset):
    def __init__(self, dataset, processor, img_dir):
        self.dataset = dataset
        self.processor = processor
        self.img_dir = img_dir

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        data = self.dataset[idx]
        image_path = os.path.join(self.img_dir, data["img"])
        image = Image.open(image_path).convert("RGB")
        text = data["text"] + " " + data["caption"]  # Combine OCR text + caption
        label = torch.tensor(data["label"], dtype=torch.float32)

        # Preprocess with SigLIP
        inputs = self.processor(text=text, images=image, return_tensors="pt", padding=True)
        inputs["label"] = label
        return inputs

# Load dataset
img_dir = "/content/dataset/data"
model_name = "google/siglip-base-patch16-256"
processor = AutoProcessor.from_pretrained(model_name)
dataset = MemeDataset(dataset, processor, img_dir)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)


In [ ]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

for epoch in range(5):  # Adjust as needed
    for batch in dataloader:
        optimizer.zero_grad()

        inputs = {k: v.squeeze(0).to("cuda") for k, v in batch.items() if k != "label"}
        labels = batch["label"].to("cuda")

        preds = model(inputs).squeeze()
        loss = criterion(preds, labels)

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")
